<a href="https://colab.research.google.com/github/newb-dev-1008/Human-PokeDex/blob/master/Human_PokeDex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade imutils

Requirement already up-to-date: imutils in /usr/local/lib/python3.7/dist-packages (0.5.4)


In [2]:
!pip install opencv-contrib-python

# **Extract embeddings from face dataset**

In [4]:
# Importing all necessary packages

from imutils import paths
import numpy as np
import argparse
import imutils
import pickle
import time
import cv2
import os

In [ ]:
# Load face detector

print("Loading face detector...")

protoPath = os.path.sep.join(['/content', 'deploy.prototxt'])
modelPath = os.path.sep.join(['/content', 'res10_300x300_ssd_iter_140000.caffemodel'])

detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)


# Load face recognizer

print("\nLoading face recognizer...")
embedder = cv2.dnn.readNetFromTorch('/content/openface_nn4.small2.v1.t7')

In [ ]:
# Entering paths to our images dataset

print('\nQuantifying faces...')
imagePaths = list(paths.list_images('/content/drive/MyDrive/Open Lab/Datasets'))

knownEmbeddings = []
knownNames = []

# Total number of faces
total = 0

In [ ]:
# Loop over all image paths

for (i, imagePath) in enumerate(imagePaths):
  print("[INFO] processing image {}/{}".format(i + 1, len(imagePaths)))           # Extract the person name from the image path
  name = imagePath.split(os.path.sep)[-2]

  image = cv2.imread(imagePath)                                                   # Load the image
  image = imutils.resize(image, width=600)                                        # Resize to (600, 600)
  (h, w) = image.shape[:2]                                                        # Store image dimensions
  imageBlob = cv2.dnn.blobFromImage(                                              # Construct a blob from the image
  	cv2.resize(image, (300, 300)), 1.0, (300, 300),
  	(104.0, 177.0, 123.0), swapRB=False, crop=False)

  detector.setInput(imageBlob)                                                    # Face Detector to localize face in an image
  detections = detector.forward()       

  if len(detections) > 0:                                                         # Ensure at least one face was found
		i = np.argmax(detections[0, 0, :, 2])                                         # Bounding box with largest probability
		confidence = detections[0, 0, i, 2]        

    if confidence > args["confidence"]:                                           # Filtering weak detections
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])                     # Coordinates for bounding box of face
			(startX, startY, endX, endY) = box.astype("int")
			
			face = image[startY:endY, startX:endX]                                      # Dimensions of ROI (Region of Interest)
			(fH, fW) = face.shape[:2]
			
			if (fW < 20 or fH < 20):                                                    # Face height and width need to be above a threshold
				continue    

      faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96), (0, 0, 0), 
                                       swapRB=True, crop=False)                   # Create blob

      embedder.setInput(faceBlob)
      vec = embedder.forward()

      knownNames.append(name)                                                     # Append name to list
      knownEmbeddings.append(vec.flatten())                                       # Append flattened embedding to list
      total += 1                                                

In [ ]:
# Save (pickle) the embeddings and the names

print ('Serializing {} encodings:\n'.format(total))

data = {'embeddings': knownEmbeddings, 'names': knownNames}                       # Saved as a dictionary/ HashMap
f = open('/content/drive/MyDrive/Open Lab/Output Embeddings', 'wb')
f.write(pickle.dumps(data))                                                       # Stored as a ByteStream
f.close()